# Leaky bucket algorithm

В ведро фиксированного размера поступают запросы клиентов. Запрос отклоняется, если не помещается в ведро. Принятый запрос кладётся в ведро и передается на обработку в порядке очереди (FIFO). Ведро наполняется запросами, приходящими, вообще говоря, неравномерно по времени. Запросы достаются из ведра с фиксированной частотой.

# Nginx burst rate limitting

```
limit_req_zone key zone=name:size rate=rate [sync];

limit_req zone=name [burst=number] [nodelay | delay=number];
```

**limit_req_zone** задаёт имя, размер, и частоту общей для worker процессов зоны, которая хранит состояния различных ключей.

Чтобы применить определённые ограничения в блоке server или location используется директива **limit_req**.

In [8]:
class RequestLimiter:
    def __init__(self, rate, burst=0, delay=float('inf')):
        "rate измеряется в r/s, nodelay по умолчанию (в nginx delay=0)"
        
        self.rate = rate
        self.burst = burst
        self.delay = delay

        self.excess = 0
        self.last_request_time = -1

    def get_request_delay(self, request_time):
        time_delta = request_time - self.last_request_time
        assert time_delta >= 0

        excess = max(0, round(self.excess - time_delta * self.rate + 1, 4))
        if excess > self.burst:
            raise RuntimeError("Слишком много запросов")

        self.excess = excess
        self.last_request_time = request_time

        if excess <= self.delay:
            return 0
        else:
            return round((excess - self.delay) / self.rate, 4)
        
def try_requests(limiter, request_times):
    for i, request_time in enumerate(request_times, 1):
        print(f"Запрос {i} пришёл в {request_time}с"б)
        try:
            delay = limiter.get_request_delay(request_time)
            print(f"Запрос {i} обработан в {request_time + delay}с" \
                  f" (задержка {delay}с)")
        except RuntimeError:
            print(f"Запрос {i} отклонён.")
        print()

### Basic Rate Limiting

Пример конфигурации

```
limit_req_zone $binary_remote_addr zone=mylimit:10m rate=10r/s;
 
server {
    location /login/ {
        limit_req zone=mylimit;
        
        proxy_pass http://my_upstream;
    }
}
```

Здесь ключ это ip адрес клиента. Частота равна 1 запрос в 100 миллисекунд.

В данном случае если с одного ip адреса посылаются два запроса к /login/, время прихода которых отличается менее чем на 100 миллисекунд, то второй запрос будет отклонён.

Зона mylimit хранит состояния для всех различных ip адресов, обращающихся к /login/. Если при попытке добавить новое состояние памяти не хватает, удаляется самое старое состояние. Если после удаления памяти всё равно не хватает запрос отклоняется. Для предотвращения заполнения памяти, при добавлении нового состояния, удаляются не более 2ух состояний, которые не использовались минуту.

При такой конфигурации очень много запросов будут отклонятся.

In [9]:
try_requests(
    RequestLimiter(10),
    [0, 0.1, 0.19, 0.2, 0.2, 0.25, 0.3]
)

Запрос 1 пришёл в 0с
Запрос 1 обработан в 0с (задержка 0с)

Запрос 2 пришёл в 0.1с
Запрос 2 обработан в 0.1с (задержка 0с)

Запрос 3 пришёл в 0.19с
Запрос 3 отклонён.

Запрос 4 пришёл в 0.2с
Запрос 4 обработан в 0.2с (задержка 0с)

Запрос 5 пришёл в 0.2с
Запрос 5 отклонён.

Запрос 6 пришёл в 0.25с
Запрос 6 отклонён.

Запрос 7 пришёл в 0.3с
Запрос 7 обработан в 0.3с (задержка 0с)



### Bursts Rate Limiting

Пример конфигурации

```
location /login/ {
    limit_req zone=mylimit burst=20;
 
    proxy_pass http://my_upstream;
}
```

Параметр **burst** задаёт размер очереди, куда будут помещаться некоторые из отклоняемых в предыдущей модели запросы (по умолчанию 0).

Будем называть запрос excessive, если на момент его прихода очередь не пуста или он пришёл слишком рано.

Теперь excessive запросы отклоняются, только если очередь уже заполнена. NGINX освобождает один слот очереди и отправляет на обработку соответствующий запрос с заданной частотой.

Используя очередь запросов, мы позволяем клиентам посылать много запросов сразу. Проблемой является высокая задержка, которая может достигать $\frac{burst}{rate}$ (2 секунды в нашем примере).

In [10]:
try_requests(
    RequestLimiter(1, 2, 0),
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]
)

Запрос 1 пришёл в 1с
Запрос 1 обработан в 1с (задержка 0с)

Запрос 2 пришёл в 1с
Запрос 2 обработан в 2.0с (задержка 1.0с)

Запрос 3 пришёл в 1с
Запрос 3 обработан в 3.0с (задержка 2.0с)

Запрос 4 пришёл в 1с
Запрос 4 отклонён.

Запрос 5 пришёл в 2с
Запрос 5 обработан в 4.0с (задержка 2.0с)

Запрос 6 пришёл в 2с
Запрос 6 отклонён.

Запрос 7 пришёл в 2с
Запрос 7 отклонён.

Запрос 8 пришёл в 2с
Запрос 8 отклонён.

Запрос 9 пришёл в 3с
Запрос 9 обработан в 5.0с (задержка 2.0с)

Запрос 10 пришёл в 3с
Запрос 10 отклонён.

Запрос 11 пришёл в 3с
Запрос 11 отклонён.

Запрос 12 пришёл в 3с
Запрос 12 отклонён.



### Queueing with No Delay

Пример конфигурации

```
location /login/ {
    limit_req zone=mylimit burst=20 nodelay;
 
    proxy_pass http://my_upstream;
}
```

Параметр **nodelay** убирает задержку, связанную с ожиданием в очереди. Теперь, если запрос принят, то он будет отправлен на обработку сразу же. Освобождение слотов очереди происходит как и ранее с заданной частотой.

Таким образом мы убрали задержку, но сохранили ограничение на число обрабатываемых запросов за относительно большой промежуток времени. Однако, возможна перегрузка серверов, к которым одновременно может прийти на обработку **burst + 1** запросов с одного ip адреса.

In [11]:
try_requests(
    RequestLimiter(1, 2),
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]
)

Запрос 1 пришёл в 1с
Запрос 1 обработан в 1с (задержка 0с)

Запрос 2 пришёл в 1с
Запрос 2 обработан в 1с (задержка 0с)

Запрос 3 пришёл в 1с
Запрос 3 обработан в 1с (задержка 0с)

Запрос 4 пришёл в 1с
Запрос 4 отклонён.

Запрос 5 пришёл в 2с
Запрос 5 обработан в 2с (задержка 0с)

Запрос 6 пришёл в 2с
Запрос 6 отклонён.

Запрос 7 пришёл в 2с
Запрос 7 отклонён.

Запрос 8 пришёл в 2с
Запрос 8 отклонён.

Запрос 9 пришёл в 3с
Запрос 9 обработан в 3с (задержка 0с)

Запрос 10 пришёл в 3с
Запрос 10 отклонён.

Запрос 11 пришёл в 3с
Запрос 11 отклонён.

Запрос 12 пришёл в 3с
Запрос 12 отклонён.



### Two-Stage Rate Limiting

Пример конфигурации

```
location /login/ {
    limit_req zone=mylimit burst=20 delay=10;
 
    proxy_pass http://my_upstream;
}
```

Данный подход комбинирует два предыдущих. Первые **delay** запросов в очереди передаются на обработку без задержки, остальные **burst - delay** запросов очереди передаются на обработку с заданной частотой.

Таким образом мы обеспечиваем быструю обработку группы запросов размера **delay + 1**, и принимаем ещё **burst - delay** запросов на обработку с задержкой.

In [12]:
try_requests(
    RequestLimiter(1, 2, 1),
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]
)

Запрос 1 пришёл в 1с
Запрос 1 обработан в 1с (задержка 0с)

Запрос 2 пришёл в 1с
Запрос 2 обработан в 1с (задержка 0с)

Запрос 3 пришёл в 1с
Запрос 3 обработан в 2.0с (задержка 1.0с)

Запрос 4 пришёл в 1с
Запрос 4 отклонён.

Запрос 5 пришёл в 2с
Запрос 5 обработан в 3.0с (задержка 1.0с)

Запрос 6 пришёл в 2с
Запрос 6 отклонён.

Запрос 7 пришёл в 2с
Запрос 7 отклонён.

Запрос 8 пришёл в 2с
Запрос 8 отклонён.

Запрос 9 пришёл в 3с
Запрос 9 обработан в 4.0с (задержка 1.0с)

Запрос 10 пришёл в 3с
Запрос 10 отклонён.

Запрос 11 пришёл в 3с
Запрос 11 отклонён.

Запрос 12 пришёл в 3с
Запрос 12 отклонён.

